In [ ]:
import os
import sys
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import random
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

def main():
    # ============================
    # Step 0: Get input CSV filename
    # ============================
    filename = input("").strip()
    file_path = os.path.join(sys.path[0], filename)

    # ============================
    # Step 1: Read CSV file
    # ============================
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        sys.exit(1)

    print("First 5 rows from the file:")
    print(df.head())
    print()

    # ============================
    # Step 2: Prepare train/test split
    # ============================
    train_df = df.copy()
    test_df = df.copy()

    # ============================
    # Step 3: Clean 'clean_text' column
    # ============================
    train_df['clean_text'] = train_df['clean_text'].fillna('')
    test_df['clean_text'] = test_df['clean_text'].fillna('')

    # Remove rows with empty text
    train_df = train_df[train_df['clean_text'].str.strip() != '']
    test_df = test_df[test_df['clean_text'].str.strip() != '']

    # Lowercase all text
    train_df['clean_text'] = train_df['clean_text'].str.lower()
    test_df['clean_text'] = test_df['clean_text'].str.lower()

    # ============================
    # Step 4: Encode labels and create y_true AFTER cleaning
    # ============================
    label_mapping = {label: idx for idx, label in enumerate(df['sentiment'].unique())}
    train_df['sentiment_encoded'] = train_df['sentiment'].map(label_mapping)
    test_df['sentiment_encoded'] = test_df['sentiment'].map(label_mapping)
    y_true = test_df['sentiment_encoded']  # must be AFTER cleaning

    # ============================
    # Step 5: Create a mock fastText predictor
    # ============================
    random.seed(42)
    def fasttext_predict_mock(text):
        return random.choice(df['sentiment'].unique())
    test_df['pred_ft'] = test_df['clean_text'].apply(fasttext_predict_mock)

    # ============================
    # Step 6: Predict multi-class data with sklearn
    # ============================
    multi_pipeline = Pipeline([
        ("tfidf", TfidfVectorizer(max_features=2000)),
        ("clf", MultinomialNB())
    ])
    multi_pipeline.fit(train_df["clean_text"], train_df["sentiment_encoded"])
    test_df['pred_sklearn'] = multi_pipeline.predict(test_df["clean_text"])

    # ============================
    # Step 7: Create a mock GenAI predictor
    # ============================
    labels = df['sentiment'].unique().tolist()
    random.seed(42)
    def genai_predict_mock(text):
        return random.choice(labels)
    test_df['pred_genai'] = test_df['clean_text'].apply(genai_predict_mock)

    # ============================
    # Step 8: Evaluate all three models
    # ============================
    ft_acc   = accuracy_score(y_true, test_df['pred_ft'].map(label_mapping))
    sk_acc   = accuracy_score(y_true, test_df['pred_sklearn'])
    gen_acc  = accuracy_score(y_true, test_df['pred_genai'].map(label_mapping))

    print("fastText Accuracy: ", round(ft_acc, 4))
    print("sklearn Accuracy: ", round(sk_acc, 4))
    print("GenAI Accuracy:   ", round(gen_acc, 4))
    print()

    # ============================
    # Step 9: Build alignment comparison table
    # ============================
    test_df['agree_ft_sk']   = (test_df['pred_ft'] == test_df['pred_sklearn'].map({v:k for k,v in label_mapping.items()}))
    test_df['agree_ft_gen']  = (test_df['pred_ft'] == test_df['pred_genai'])
    test_df['agree_sk_gen']  = (test_df['pred_sklearn'].map({v:k for k,v in label_mapping.items()}) == test_df['pred_genai'])

    alignment_results = test_df[['agree_ft_sk', 'agree_ft_gen', 'agree_sk_gen']].mean()
    print("Alignment Results:")
    print(alignment_results)
    print()

    # ============================
    # Step 10: Inspect mock fastText predictions
    # ============================
    test_df['ft_pred_raw'] = test_df['clean_text'].apply(lambda x: (fasttext_predict_mock(x), random.random()))
    print("Sample fastText raw predictions:")
    print(test_df[['clean_text', 'ft_pred_raw']].head(5))
    print()

    # ============================
    # Step 11: Interpretation
    # ============================
    print(" Where fastText > sklearn?")
    print(test_df[(test_df['agree_ft_gen']) & (~test_df['agree_ft_sk'])].head(3))
    print()

    print(" Where sklearn > fastText?")
    print(test_df[(~test_df['agree_ft_sk']) & (test_df['agree_sk_gen'])].head(3))
    print()

    print(" Where GenAI > both?")
    print(test_df[
        (test_df['pred_genai'].map(label_mapping) == y_true) &
        (test_df['pred_sklearn'] != y_true) &
        (test_df['pred_ft'].map(label_mapping) != y_true)
    ].head(3))


if __name__ == "__main__":
    main()


[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                             
2020-01-02  72.468277  72.528597  71.223274  71.476615  135480400
2020-01-03  71.763725  72.523754  71.539337  71.696167  146322800
2020-01-06  72.335556  72.374162  70.634539  70.885472  118387200
2020-01-07  71.995369  72.600975  71.775804  72.345220  108872000
2020-01-08  73.153481  73.455080  71.698566  71.698566  132079200
